In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from config import Config_list
from config import config_dict



In [3]:
# restart kernel after running config
len(Config_list)

11

In [4]:
from pathlib import Path
import matplotlib.pyplot as plt

# create df and label target label new
df_list = []
csv_list = []
for i in range(len(Config_list)):
    
    if Config_list[i][1]['TYPE'] == "BINARY":
        if Config_list[i][1]["DATA_SET"] not in csv_list:
            temp_df = pd.read_csv(Config_list[i][1]["DATA_SET"])
            csv_list.append(Config_list[i][1]["DATA_SET"])
            temp_df = temp_df.drop(columns=Config_list[i][1]['DROP'])
            temp_df= temp_df.rename(columns={temp_df[Config_list[i][1]['TARGET_LABEL']].name : 'TARGET_LABEL'})
            df_list.append(temp_df)
        
eval_df = pd.DataFrame(csv_list, columns=['Data_Set'])
eval_df["Target_1_pos"] = pd.Series()
eval_df["Target_2_neg"] = pd.Series()
eval_df["IB_Ratio"] = pd.Series()
eval_df["Size"] = pd.Series()

eval_df

C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future versi

,Data_Set,Target_1_pos,Target_2_neg,IB_Ratio,Size
0,../data/churn_prob_out_35.csv,NaN,NaN,NaN,NaN
1,../data/german_credit_codiert.csv,NaN,NaN,NaN,NaN
2,../data/compas.csv,NaN,NaN,NaN,NaN
3,../data/mushroom.csv,NaN,NaN,NaN,NaN
4,../data/fraud_detection.csv,NaN,NaN,NaN,NaN


In [5]:
for frame in range(len(df_list)):
    metric = df_list[frame]["TARGET_LABEL"].value_counts()
    if metric[0] > metric[1]:
        eval_df["Target_1_pos"].iloc[frame] = metric[1]
        eval_df["Target_2_neg"].iloc[frame] = metric[0]
    else:
        eval_df["Target_1_pos"].iloc[frame] = metric[0]
        eval_df["Target_2_neg"].iloc[frame] = metric[1]
    df_size = len(df_list[frame])
    eval_df["IB_Ratio"].iloc[frame] = eval_df["Target_1_pos"].iloc[frame]/eval_df["Target_2_neg"].iloc[frame]
    eval_df["Size"].iloc[frame]    = df_size
    print(metric)
    

F    1076
T     723
Name: TARGET_LABEL, dtype: int64
1    700
0    300
Name: TARGET_LABEL, dtype: int64
0    3960
1    3250
Name: TARGET_LABEL, dtype: int64
e    4208
p    3916
Name: TARGET_LABEL, dtype: int64
0    284315
1       492
Name: TARGET_LABEL, dtype: int64


C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [6]:
eval_df

,Data_Set,Target_1_pos,Target_2_neg,IB_Ratio,Size
0,../data/churn_prob_out_35.csv,723.0,1076.0,0.671933,1799.0
1,../data/german_credit_codiert.csv,300.0,700.0,0.428571,1000.0
2,../data/compas.csv,3250.0,3960.0,0.820707,7210.0
3,../data/mushroom.csv,3916.0,4208.0,0.930608,8124.0
4,../data/fraud_detection.csv,492.0,284315.0,0.001730,284807.0
